In [1]:
import math
import random
import unittest

# ------------------------ Implementation ------------------------
def daxpy(a, x, y):
    """Compute d = a*x + y (elementwise). Raises ValueError on length mismatch."""
    if len(x) != len(y):
        raise ValueError("x and y must have the same length")
    return [a*xi + yi for xi, yi in zip(x, y)]

# ------------------------ Test Helpers -------------------------
def allclose_vec(v1, v2, atol=1e-12, rtol=1e-12):
    if len(v1) != len(v2):
        return False
    for a, b in zip(v1, v2):
        if math.isfinite(a) and math.isfinite(b):
            if abs(a-b) > atol + rtol*max(abs(a), abs(b)):
                return False
        else:
            # strict handling for non-finite
            if not (math.isnan(a) and math.isnan(b)) and a != b:
                return False
    return True

# ------------------------ Unit Tests ---------------------------
class TestDaxpy(unittest.TestCase):
    def test_small_known(self):
        self.assertEqual(daxpy(3.0, [0.1,0.1,0.1], [7.1,7.1,7.1]), [7.4,7.4,7.4])

    def test_zero_scalar(self):
        self.assertEqual(daxpy(0.0, [1.0,2.0,3.0], [4.0,5.0,6.0]), [4.0,5.0,6.0])

    def test_zero_vectors(self):
        n = 5
        self.assertEqual(daxpy(2.0, [0.0]*n, [0.0]*n), [0.0]*n)

    def test_length_mismatch(self):
        with self.assertRaises(ValueError):
            daxpy(2.0, [1.0,2.0], [1.0])

    def test_random_reproducible(self):
        rng = random.Random(123)
        n = 1000
        a = 1.7
        x = [rng.uniform(-10, 10) for _ in range(n)]
        y = [rng.uniform(-10, 10) for _ in range(n)]
        ref = [a*xi + yi for xi, yi in zip(x, y)]
        out = daxpy(a, x, y)
        self.assertTrue(allclose_vec(out, ref))

    def test_nan_inf_passthrough(self):
        a = 2.0
        x = [1.0, math.nan, math.inf, -math.inf]
        y = [3.0, 1.0, 5.0, 6.0]
        out = daxpy(a, x, y)   # [5.0, nan, inf, -inf]
        self.assertTrue(math.isclose(out[0], 5.0))
        self.assertTrue(math.isnan(out[1]))
        self.assertTrue(math.isinf(out[2]) and out[2] > 0)
        self.assertTrue(math.isinf(out[3]) and out[3] < 0)

    def test_numpy_parity(self):
        try:
            import numpy as np
        except Exception:
            self.skipTest("NumPy not installed")
        a = 3.0
        x = [0.1, -2.0, 5.5, 0.0]
        y = [7.1,  1.0, 2.5, 8.0]
        ref = (a*np.array(x) + np.array(y)).tolist()
        out = daxpy(a, x, y)
        self.assertTrue(allclose_vec(out, ref))

# ------------------------ Runner -------------------------------
suite = unittest.defaultTestLoader.loadTestsFromTestCase(TestDaxpy)
unittest.TextTestRunner(verbosity=2).run(suite)


test_length_mismatch (__main__.TestDaxpy.test_length_mismatch) ... ok
test_nan_inf_passthrough (__main__.TestDaxpy.test_nan_inf_passthrough) ... ok
test_numpy_parity (__main__.TestDaxpy.test_numpy_parity) ... ok
test_random_reproducible (__main__.TestDaxpy.test_random_reproducible) ... ok
test_small_known (__main__.TestDaxpy.test_small_known) ... FAIL
test_zero_scalar (__main__.TestDaxpy.test_zero_scalar) ... ok
test_zero_vectors (__main__.TestDaxpy.test_zero_vectors) ... ok

FAIL: test_small_known (__main__.TestDaxpy.test_small_known)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/tmp/ipykernel_882032/3085391614.py", line 29, in test_small_known
    self.assertEqual(daxpy(3.0, [0.1,0.1,0.1], [7.1,7.1,7.1]), [7.4,7.4,7.4])
    ~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
AssertionError: Lists differ: [7.3999999999999995, 7.3999999999999995, 7.3999999999999995] != [7.4, 7.4, 7.4]

First diff

<unittest.runner.TextTestResult run=7 errors=0 failures=1>